#Preprocessing

###Notebook with the preprocessing tasks for the project: MyProject

##### - Model input:dbfs:/Users/ju195@cummins.com/teste/MyProject/model/input

##### - Model output:dbfs:/Users/ju195@cummins.com/teste/MyProject/model/output

##### - Model dbfs:/Users/ju195@cummins.com/teste/MyProject/model/artifacts

##1 Discovering Butterfree - Feature Set Basics

Welcome to **Discovering Butterfree** tutorial series!

This first tutorial will cover some basics of Butterfree library and you learn how to create your first feature set :rocket: :rocket:

Before diving into the tutorial make sure you have a basic understanding of these main data concepts: **features**, **feature sets** and the **"Feature Store Architecture"**, you can read more about this [here]().

## Library Basics:

Buterfree's main objective is to make feature engineering easy. The library provides a high-level API for declarative feature definitions. But behind these abstractions, Butterfree is essentially an **ETL (Extract - Transform - Load)** framework, so this reflects in terms of the organization of the project.

### Extract

`from butterfree.extract import ...`

Module with the entities responsible for extracting data into the pipeline. The module provides the following tools:

* `readers`: data connectors. Currently Butterfree provides readers for files, tables registered in Spark Hive metastore, and Kafka topics.


* `pre_processing`: a utility tool for making some transformations or re-arrange the structure of the reader's input data before the feature engineering.


* `source`: a composition of `readers`. The entity responsible for merging datasets coming from the defined readers into a single dataframe input for the `Transform` stage.

### Transform

`from butterfree.transform import ...`

The main module of the library, responsible for feature engineering, in other words, all the transformations on the data. The module provides the following main tools:

* `features`: the entity that defines what a feature is. Holds a transformation and metadata about the feature.


* `transformations`: provides a set of components for transforming the data, with the possibility to use Spark native functions, aggregations, SQL expressions and others. 


* `feature_set`: an entity that defines a feature set. Holds features and the metadata around it.


### Load

`from butterfree.load import ...`

The module is responsible for saving the data in some data storage. The module provides the following tools:

* `writers`: provide connections to data sources to write data. Currently Butterfree provides ways to save data on S3 registered as tables Spark Hive metastore and to Cassandra DB.


* `sink`: a composition of writers. The entity responsible for triggering the writing jobs on a set of defined writers

### Pipelines

Pipelines are responsible for integrating all other modules (`extract`, `transform`, `load`) in order to define complete ETL jobs from source data to data storage destination.

`from butterfree.pipelines import ...`

* `feature_set_pipeline`: defines an ETL pipeline for creating feature sets.

## Install Butterfree:

In [4]:
dbutils.library.installPyPI("butterfree")

Out[1]: True

## Example:
Simulating the following scenario:

- We want to create a feature set with features about houses for rent (listings).

- We are interested in houses only for the **Kanto** region.

We have two sets of data:

- Table: `listing_events`. Table with data about events of house listings.
- File: `region.json`. Static file with data about the cities and regions.

Our desire is to have result dataset with the following schema:

| id | timestamp | rent | rent_over_area | bedrooms | bathrooms | area | bedrooms_over_area | bathrooms_over_area | latitude | longitude | h3 | city | region 
| - | - | - | - | - | - | - | - | - | - | - | - | - | - |
| int | timestamp | float | float | int | int | float | float | float | double | double | string | string | string |

For more information about H3 geohash click [here]()

The following code blocks will show how to generate this feature set using Butterfree library:

In [6]:
# setup spark
from pyspark import SparkContext, SparkConf
from pyspark.sql import session
import  pyspark.sql.functions as F
# butterfree spark client
from butterfree.clients import SparkClient

spark_client = SparkClient()


### Showing test data

In [8]:
listing_evengs_df = spark.read.json(f"{path}/examples/data/listing_events.json")
listing_evengs_df.createOrReplaceTempView("listing_events")  # creating listing_events table

print(">>> listing_events table:")
listing_evengs_df.toPandas()

In [9]:
print(">>> region.json file:")
spark.read.json(f"{path}/examples/data/region.json").toPandas()

### Extract

- For the extract part, we need the `Source` entity and the `FileReader` and `TableReader` for the data we have.
- We need to declare a query with the rule for joining the results of the readers too.
- As proposed in the problem we can filter the region dataset to get only **Kanto** region.

In [11]:
from butterfree.extract import Source
from butterfree.extract.readers import FileReader, TableReader
from butterfree.extract.pre_processing import filter

readers = [
    TableReader(id="listing_events", table="listing_events",),
    FileReader(id="region", path=f"{path}/examples/data/region.json", format="json",).with_(
        transformer=filter, condition="region == 'Kanto'"
    ),
]

query = """
select
    listing_events.*,
    region.city,
    region.lat,
    region.lng,
    region.region
from
    listing_events
    join region
      on listing_events.region_id = region.id
"""

source = Source(readers=readers, query=query)

In [12]:
# showing source result

source_df = source.construct(spark_client)
source_df.toPandas()

### Transform
- At the transform part, a set of `Feature` objects is declared.
- An Instance of `FeatureSet` is used to hold the features.
- A `FeatureSet` can only be created when it is possible to define a unique tuple formed by key columns and a time reference. This is an **architectural requirement** for the data. So least one `KeyFeature` and one `TimestampFeature` is needed.
- Every `Feature` needs a unique name, a description, and a data-type definition.

In [14]:
from butterfree.transform import FeatureSet
from butterfree.transform.features import Feature, KeyFeature, TimestampFeature
from butterfree.transform.transformations import SQLExpressionTransform
from butterfree.transform.transformations.h3_transform import H3HashTransform
from butterfree.constants import DataType

keys = [
    KeyFeature(
        name="id",
        description="Unique identificator code for houses.",
        dtype=DataType.BIGINT,
    )
]

# from_ms = True because the data originally is not in a Timestamp format.
ts_feature = TimestampFeature(from_ms=True)

features = [
    Feature(
        name="rent",
        description="Rent value by month described in the listing.",
        dtype=DataType.FLOAT,
    ),
    Feature(
        name="rent_over_area",
        description="Rent value by month divided by the area of the house.",
        transformation=SQLExpressionTransform("rent / area"),
        dtype=DataType.FLOAT,
    ),
    Feature(
        name="bedrooms",
        description="Number of bedrooms of the house.",
        dtype=DataType.INTEGER,
    ),
    Feature(
        name="bathrooms",
        description="Number of bathrooms of the house.",
        dtype=DataType.INTEGER,
    ),
    Feature(
        name="area",
        description="Area of the house, in squared meters.",
        dtype=DataType.FLOAT,
    ),
    Feature(
        name="bedrooms_over_area",
        description="Number of bedrooms divided by the area.",
        transformation=SQLExpressionTransform("bedrooms / area"),
        dtype=DataType.FLOAT,
    ),
    Feature(
        name="bathrooms_over_area",
        description="Number of bathrooms divided by the area.",
        transformation=SQLExpressionTransform("bathrooms / area"),
        dtype=DataType.FLOAT,
    ),
    Feature(
        name="latitude",
        description="House location latitude.",
        from_column="lat",  # arg from_column is needed when changing column name
        dtype=DataType.DOUBLE,
    ),
    Feature(
        name="longitude",
        description="House location longitude.",
        from_column="lng",
        dtype=DataType.DOUBLE,
    ),
    Feature(
        name="h3",
        description="H3 hash geohash.",
        transformation=H3HashTransform(
            h3_resolutions=[10], lat_column="latitude", lng_column="longitude",
        ),
        dtype=DataType.STRING,
    ),
    Feature(name="city", description="House location city.", dtype=DataType.STRING,),
    Feature(
        name="region",
        description="House location region.",
        dtype=DataType.STRING,
    ),
]

feature_set = FeatureSet(
    name="house_listings",
    entity="house",  # entity: to which "business context" this feature set belongs
    description="Features describring a house listing.",
    keys=keys,
    timestamp=ts_feature,
    features=features,
)

In [15]:
# showing feature set result

feature_set_df = feature_set.construct(source_df, spark_client)
feature_set_df.toPandas()

### Load

- For the load part we need `Writer` instances and a `Sink`.
- writers define where to load the data.
- The `Sink` gets the transformed data (feature set) and trigger the load to all the defined writers.
- `debug_mode` will create a temporary view instead of trying to write in a real data store.

In [17]:
from butterfree.load.writers import (
    HistoricalFeatureStoreWriter,
    OnlineFeatureStoreWriter,
)
from butterfree.load import Sink

writers = [HistoricalFeatureStoreWriter(debug_mode=True), OnlineFeatureStoreWriter(debug_mode=True)]
sink = Sink(writers=writers)

## Pipeline

- The `Pipeline` entity wraps all the other defined elements.
- `run` command will trigger the execution of the pipeline, end-to-end.

In [19]:
from butterfree.pipelines import FeatureSetPipeline

pipeline = FeatureSetPipeline(source=source, feature_set=feature_set, sink=sink)

In [20]:
result_df = pipeline.run()

### Showing the results

In [22]:
print(">>> Historical Feature house_listings feature set table:")
spark.table("historical_feature_store__house_listings").orderBy(
    "id", "timestamp"
).toPandas()

In [23]:
print(">>> Online Feature house_listings feature set table:")
spark.table("online_feature_store__house_listings").orderBy("id", "timestamp").toPandas()

- We can see that we were able to create all the desired features in an easy way
- The **historical feature set** holds all the data, and we can see that it is partitioned by year, month and day (columns added in the `HistoricalFeatureStoreWriter`)
- In the **online feature set** there is only the latest data for each id